In [20]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
def GM11(x,n):
    '''
    灰色预测
    x：序列，numpy对象
    n:需要往后预测的个数
    '''
    x1 = x.cumsum()#一次累加  
    z1 = (x1[:len(x1) - 1] + x1[1:])/2.0#紧邻均值  
    z1 = z1.reshape((len(z1),1))  
    B = np.append(-z1,np.ones_like(z1),axis=1)  
    Y = x[1:].reshape((len(x) - 1,1))
    #a为发展系数 b为灰色作用量
    [[a],[b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)#计算参数  
    result = (x[0]-b/a)*np.exp(-a*(n-1))-(x[0]-b/a)*np.exp(-a*(n-2))  
    S1_2 = x.var()#原序列方差
    e = list()#残差序列
    for index in range(1,x.shape[0]+1):
        predict = (x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2))
        e.append(x[index-1]-predict)
    S2_2 = np.array(e).var()#残差方差
    C = S2_2/S1_2#后验差比
    if C<=0.35:
        assess = '后验差比<=0.35，模型精度等级为好'
    elif C<=0.5:

        assess = '后验差比<=0.5，模型精度等级为合格'

    elif C<=0.65:
        assess = '后验差比<=0.65，模型精度等级为勉强'
    else:
        assess = '后验差比>0.65，模型精度等级为不合格'
    #预测数据
    predict = list()
    for index in range(x.shape[0]+1,x.shape[0]+n+1):
        predict.append((x[0]-b/a)*np.exp(-a*(index-1))-(x[0]-b/a)*np.exp(-a*(index-2)))
    predict = np.array(predict)
    return predict

In [38]:
import pandas as pd
a0=pd.read_excel(r'C:\Users\stu\Desktop\RSDATA.xlsx').values
m,n=a0.shape
yuc=[]
for i in range(n):
    yu=[]
    a1=a0[:,i]
    x=a1[1800:1950]
    y=a1[1950:2000]
    yu= GM11(x,len(y))
    yuc.append(yu)

In [39]:
zs=np.array(a0[:,2])[1950:2000]
yuce=yuc[2]
error = []
#print(zs,yuce)

for i in range(len(zs)):
    error.append(zs[i] -yuce[i])
#print("Errors: ", error)
#print(error)
squaredError = []
absError = []
for val in error:    
    squaredError.append(val * val)#target-prediction之差平方     
    absError.append(abs(val))#误差绝对值  
    #print("Square Error: ", squaredError)
    #print("Absolute Value of Error: ", absError)    
print("MSE = ", sum(squaredError) / len(squaredError))#均方误差MSE    
from math import sqrt
print("RMSE = ", sqrt(sum(squaredError) / len(squaredError)))#均方根误差RMSE
print("MAE = ", sum(absError) / len(absError))#平均绝对误差MAE

MSE =  617373.9982970725
RMSE =  785.7315052211362
MAE =  707.6680507469879


In [6]:
import xlsxwriter   #导入模块

workbook = xlsxwriter.Workbook('GM.xlsx')     #新建excel表
worksheet= workbook.add_worksheet('sheet1')       #新建sheet（sheet的名称为"sheet1"）
headings = ['cpu1','cpu2','load1','load2']     #设置表头
worksheet.write_row('A1',headings)
worksheet.write_column('A2',np.array(yuc[0]))
worksheet.write_column('B2',np.array(yuc[1]))
worksheet.write_column('C2',np.array(yuc[2])) 
worksheet.write_column('D2',np.array(yuc[3]))
##worksheet.write_column('F2',np.array(yuc[5])) 
#将数据插入到表格中 
workbook.close()          #将excel文件保存关闭，如果没有这一行运行代码会报错